In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import logging
import time
from scipy import misc 
from glob import glob
from PIL import Image
import numpy as np
import flow_vis

In [ ]:
def resize_flow(flow, new_shape):
    h, w, _ = flow.shape
    new_h, new_w = new_shape
    flow = cv2.resize(flow, (new_h, new_w), interpolation=cv2.INTER_LINEAR)
    scale_h, scale_w = h / float(new_h), w / float(new_w)
    flow[:, 0] /= scale_w
    flow[:, 1] /= scale_h
    return flow

In [ ]:
class Robust_PCA:
    def __init__(self, matrix, max_iter=100, print_freq=10):
        self.matrix = matrix # / np.linalg.norm(matrix, ord=2)
        self.max_iter = max_iter
        
        self.lambda_1 = 0.03 #4. / np.sqrt(matrix.shape[0]) # 0.3
        self.lambda_2 = 0.05 #1000. / np.sqrt(matrix.shape[0]) # 0.05 #2.0 
        
        self.tolerance = 1e-6
        self.beta = 1.25 / np.linalg.norm(self.matrix, ord=2) # ord=2
        # self.beta = np.prod(matrix.shape) / (4 * np.linalg.norm(matrix, ord=1))
        self.beta_inv = 1.0 / self.beta
        self.rho = 1.14 # 1.004
        
        self.print_freq = print_freq
        
    def frobenius_norm(self, matrix):
        return np.linalg.norm(matrix, ord='fro')
    
    def svd_threshold(self, matrix, theta):
        u, s, v = np.linalg.svd(matrix, full_matrices=False)
        return u.dot(np.diag(self.shrink(s, theta))).dot(v.conj())
    
    def shrink(self, matrix, theta):
        arg_z = np.angle(matrix)
        return np.exp(1j * arg_z) * np.maximum((np.abs(matrix) - theta), np.zeros(matrix.shape))
        
    def fit(self):
        idx = 0
        loss = np.Inf
        s_k = np.zeros(self.matrix.shape)
        l_k = np.zeros(self.matrix.shape)
        e_k = np.zeros(self.matrix.shape)
        y_k = self.matrix / (np.linalg.norm(self.matrix, ord=2))
        # y_k = np.zeros(self.matrix.shape)
        
        while (loss > self.tolerance) and (idx < self.max_iter):            
            l_k = self.svd_threshold(self.matrix - s_k - e_k + self.beta_inv * y_k, self.beta_inv)
            s_k = self.shrink(self.matrix - l_k - e_k + self.beta_inv * y_k, self.lambda_1 * self.beta_inv)
            e_k = (1.0 / (1 + 2. * self.lambda_2 * self.beta_inv)) * (self.matrix - l_k - s_k + self.beta_inv * y_k)
            y_k = y_k + self.beta * (self.matrix - l_k - s_k - e_k)
            
            self.beta = self.beta * self.rho
            self.beta_inv = 1.0 / self.beta
                        
            loss = self.frobenius_norm(self.matrix - l_k - s_k - e_k) / self.frobenius_norm(self.matrix)
            
            if idx % self.print_freq == 0:
                tmp = np.abs(self.matrix - l_k - e_k + self.beta_inv * y_k)
                print('Beta: {:6f}, Beta_inv: {:6f}'.format(self.beta, self.beta_inv))
                print('Lambda_S: {:6f}'.format(self.beta_inv * self.lambda_1))
                print('iteration: {:d}, loss: {:6f}'.format(idx, loss))
                
            idx += 1
                
        self.L = l_k
        self.S = s_k
        return l_k, s_k, e_k

In [ ]:
#generated
seq_len = 300
h, w = (10, 10)
ut_u, ut_v = np.meshgrid(np.linspace(-9, 9, 10), np.linspace(-9, 9, 10))
ur_u = np.ones((h, w)) * 2
ur_v = np.zeros((h, w))
x, y = np.meshgrid(np.linspace(0, 9, 10), np.linspace(0, 9, 10))

ut = ut_u + 1j * ut_v
ur = ur_u + 1j * ur_v

#create data
seq_flow = []
l_list = []
s_list = []
e_list = []

for n in range(1, seq_len + 1):
    vt_n = 0.5 * (1 - 2 * n / 300)
    vr_n = 0.8 * np.exp(1j * 2 * np.pi * n / 10)
    
    s_tmp = np.zeros((h, w))
    # s_tmp[:, 8] = np.random.uniform(low=-3, high=3, size=1)
    s_tmp[:, 8] = -2.7
    
    l_n = vt_n * ut + vr_n * ur
    s_n = s_tmp
    e_n = np.random.normal(0, 0.1, (h, w)) + 1j * np.random.normal(0, 0.1, (h, w))
    c_n = l_n + s_n + e_n
    
    seq_flow.append(c_n)
    l_list.append(l_n)
    s_list.append(s_n)
    e_list.append(e_n)

full_list = [seq_flow[91], l_list[91], s_list[91], e_list[91]]
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(20, 10))

for idx, ax in enumerate(axes):
    u = np.array(np.real(full_list[idx]))
    v = np.array(np.imag(full_list[idx]))
    ax.yaxis.tick_left()
    ax.xaxis.tick_top()
    ax.set_ylim((-10, 1))
    ax.set_xlim((-1, 10))
    if idx == 3:
        # Error
        ax.quiver(x.flatten(), -y.flatten(), u.flatten(), -v.flatten(), scale=0.9, 
                  units='xy', width=0.05, headlength=3, headaxislength=2, linestyle='dashed')
    elif idx == 2:
        # Sparse
        ax.quiver(x.flatten(), -y.flatten(), u.flatten(), -v.flatten(), scale=3, 
                  units='xy', width=0.05, headlength=3, headaxislength=2, linestyle='dashed')
    else:
        # Total, Lowrank
        ax.quiver(x.flatten(), -y.flatten(), u.flatten(), -v.flatten(), scale=2, 
                  units='xy', width=0.05, headlength=3, headaxislength=2, linestyle='dashed')
        
    ax.set_aspect('equal')
    
plt.show()

In [ ]:
image_flow = np.array(seq_flow).reshape(seq_len, -1) # 300 x 100
complex_flow = np.transpose(image_flow, (1, 0))

print(complex_flow.shape)

In [ ]:
r_pca_uv = Robust_PCA(complex_flow, max_iter=10000)
L_uv, S_uv, e = r_pca_uv.fit()

In [ ]:
lowrank_flow = np.transpose(L_uv, (1, 0))[91, :] # * np.linalg.norm(complex_flow, ord=1)
sparse_flow = np.transpose(S_uv, (1, 0))[91, :] # * np.linalg.norm(complex_flow, ord=1)
e = np.transpose(e, (1, 0))[10, :]

L_u = lowrank_flow.real
L_v = lowrank_flow.imag
e_u = e.real
e_v = e.imag
S_u = sparse_flow.real
S_v = sparse_flow.imag

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=1, ncols=4, figsize=(20, 15))

ax1.yaxis.tick_left()
ax1.xaxis.tick_top()
ax1.set_ylim((-10, 1)) 
ax1.set_xlim((-1, 10)) 
ax1.quiver(x.flatten(), -y.flatten(), (L_u+S_u).flatten(), -(L_v+S_v).flatten(), scale=2, 
           units='xy', width=0.05, headlength=3, headaxislength=2, linestyle='dashed')
ax1.set_aspect('equal')

ax2.yaxis.tick_left()
ax2.xaxis.tick_top()
ax2.set_ylim((-10, 1)) 
ax2.set_xlim((-1, 10)) 
ax2.quiver(x.flatten(), -y.flatten(), L_u.flatten(), -L_v.flatten(), scale=2, 
           units='xy', width=0.05, headlength=3, headaxislength=2, linestyle='dashed')
ax2.set_aspect('equal')

ax3.yaxis.tick_left()
ax3.xaxis.tick_top()
ax3.set_ylim((-10, 1)) 
ax3.set_xlim((-1, 10)) 
ax3.quiver(x.flatten(), -y.flatten(), S_u.flatten(), -S_v.flatten(), scale=3, 
           units='xy', width=0.05, headlength=3, headaxislength=2, linestyle='dashed')
ax3.set_aspect('equal')

ax4.yaxis.tick_left()
ax4.xaxis.tick_top()
ax4.set_ylim((-10, 1)) 
ax4.set_xlim((-1, 10)) 
ax4.quiver(x.flatten(), -y.flatten(), e_u.flatten(), -e_v.flatten(), scale=3, 
           units='xy', width=0.05, headlength=3, headaxislength=2, linestyle='dashed')
ax4.set_aspect('equal')